In [1]:
from libs.data_manipulation import *
from libs.feature_generation import *
from libs.dim_reduction import *
from libs.football_plots import *
from libs.clustering import *
from libs.convex_hull import *
from libs.weight_generator import *
from scipy.stats import wasserstein_distance_nd


/home/martin/.local/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (None)/charset_normalizer (2.1.0) doesn't match a supported version!
  warnings.warn(


In [2]:
england_df = compile_team_tracking_data("data", "England")


In [13]:
def normalize_positions_with_ball(df):
    # Create copies of ball x and y columns for both home and away teams
    df_normalized = df.copy()
    
    # Normalize 'home' team player positions
    home_columns_x = [col for col in df.columns if col.startswith('home') and col.endswith('_x')]
    home_columns_y = [col for col in df.columns if col.startswith('home') and col.endswith('_y')]
    
    for x_col, y_col in zip(home_columns_x, home_columns_y):
        df_normalized[x_col] = df[x_col] - df['ball_x']
        df_normalized[y_col] = df[y_col] - df['ball_y']
    
    # Normalize 'away' team player positions
    away_columns_x = [col for col in df.columns if col.startswith('away') and col.endswith('_x')]
    away_columns_y = [col for col in df.columns if col.startswith('away') and col.endswith('_y')]
    
    for x_col, y_col in zip(away_columns_x, away_columns_y):
        df_normalized[x_col] = df[x_col] - df['ball_x']
        df_normalized[y_col] = df[y_col] - df['ball_y']
    
    return df_normalized

In [31]:
one_match = extract_one_match(england_df).iloc[::48]
one_match_oringal = one_match
#one_match = normalize_positions_with_ball(one_match)
identified_corner_timestamp = 4432.84

In [32]:
one_match.iloc[37].filter(regex ="^home")


home_6_x    -13.496262
home_6_y      1.797802
home_1_x     -38.15736
home_1_y     -4.542754
home_10_x     3.410235
home_10_y     1.281452
home_11_x    13.490465
home_11_y    11.680944
home_2_x    -16.483423
home_2_y     -23.33276
home_7_x     12.318997
home_7_y    -35.212608
home_8_x     -2.864216
home_8_y    -19.845928
home_9_x      5.605203
home_9_y    -31.943056
home_5_x    -19.782419
home_5_y    -10.132314
home_12_x     -6.47533
home_12_y    13.517726
home_4_x     -7.085839
home_4_y     -2.173497
home_20_x          NaN
home_20_y          NaN
home_16_x          NaN
home_16_y          NaN
home_19_x          NaN
home_19_y          NaN
home_21_x          NaN
home_21_y          NaN
Name: 1776, dtype: object

In [33]:



identified_corner_df = one_match[one_match["Time [s]"] == identified_corner_timestamp]

func = lambda x : np.exp(-x)

inverse_identified_corner_weights = calculate_weights(identified_corner_df, fun = func)
inverse_distance_list = calculate_weights(one_match, fun= func) #Inverse proportionality to distance

In [34]:
from scipy.stats import wasserstein_distance_nd
one_match = normalize_positions_with_ball(one_match)
coordinates = one_match.filter(regex ="^home").to_numpy()





identified_corner_coordinates_numpy = identified_corner_df.to_numpy()
identified_corner_coordinates = [list(zip(row[~np.isnan(row)][::2],row[~np.isnan(row)][1::2])) for row in coordinates]
coordinates_zipped = [list(zip(row[~np.isnan(row)][::2],row[~np.isnan(row)][1::2])) for row in coordinates]


In [35]:
print(len(inverse_identified_corner_weights[0]))


11


In [36]:

#Get closest situations
distances = []
i = 0
for weights, coordinates in zip(inverse_distance_list, coordinates_zipped):
    i+=1
    if(not np.isnan(np.sum(weights)) and len(weights) == 11 and len( coordinates) == 11):
        print(inverse_identified_corner_weights[0], print(weights))
        distances.append((wasserstein_distance_nd(identified_corner_coordinates[0], coordinates, u_weights= inverse_identified_corner_weights[0], v_weights=weights), i))

[2.931672803206414e-06, 0.0, 4.822899859681319e-10, 5.34296041521568e-13, 7.057987890577919e-07, 0.9996019543773889, 0.00022281315772236757, 4.116017822100641e-07, 6.243203275409105e-06, 9.350629616769401e-09, 0.0001649303547850627]
[5.771801245076831e-15, 0.0, 2.93781040533809e-13, 3.039725848254564e-17, 1.786094787961542e-16, 1.6024673029174888e-19, 5.687662788483035e-17, 0.9999999998719951, 6.846486087765689e-16, 8.30665626442282e-17, 1.2770430746600217e-10] None
[7.738239342465963e-06, 0.9999360318410041, 1.7598265312887735e-13, 0.0, 2.8491854562082588e-08, 1.0597694390004874e-10, 4.914660874116942e-07, 5.344835511331534e-13, 5.567885742494295e-05, 3.6996678010349395e-09, 2.729793141265423e-08]
[5.771801245076831e-15, 0.0, 2.93781040533809e-13, 3.039725848254564e-17, 1.786094787961542e-16, 1.6024673029174888e-19, 5.687662788483035e-17, 0.9999999998719951, 6.846486087765689e-16, 8.30665626442282e-17, 1.2770430746600217e-10] None
[2.9783602694766592e-08, 0.9999876469403499, 5.2806317

In [37]:
indices_and_distances = sorted(distances, key = lambda t: t[0])
indices = [index for _,index in indices_and_distances]
print(indices)

[553, 952, 554, 569, 719, 139, 2490, 335, 132, 2297, 721, 724, 723, 605, 1261, 722, 1941, 1582, 1569, 2229, 953, 113, 570, 1260, 1146, 1627, 529, 58, 552, 2938, 1628, 568, 489, 1446, 115, 2574, 458, 2429, 2152, 1929, 2754, 1928, 334, 2231, 606, 1927, 2080, 589, 210, 1312, 1031, 114, 1860, 1313, 169, 1147, 1450, 1926, 1919, 2332, 1924, 140, 2367, 1920, 1923, 1925, 1921, 1743, 2760, 2368, 1918, 73, 2369, 1966, 619, 1885, 1922, 746, 1049, 138, 2370, 211, 1436, 1372, 1259, 2548, 2225, 432, 457, 635, 2755, 1274, 26, 781, 1811, 1947, 1828, 961, 480, 1499, 1944, 1917, 166, 2294, 1574, 870, 116, 1430, 2444, 617, 174, 1381, 1629, 2328, 1630, 850, 2024, 1536, 1089, 960, 2761, 496, 2352, 1067, 2295, 47, 1357, 165, 574, 2491, 2820, 1829, 1886, 2458, 1583, 119, 954, 2149, 637, 1425, 6, 2280, 1717, 2366, 2746, 2087, 803, 955, 164, 1916, 222, 590, 142, 573, 1508, 2535, 226, 575, 743, 2298, 1570, 1292, 2425, 1807, 218, 591, 777, 2531, 2302, 1127, 1730, 118, 2392, 516, 1220, 1151, 1218, 1687, 501, 24, 

In [38]:
situations = one_match.iloc[indices[:10]].index.to_numpy()


In [39]:

generate_pitches_from_start_indices(np.concatenate((situations,identified_corner_df.index.to_numpy())), one_match_oringal, "Ripleysk_k_plots",1,48)

/home/martin/anaconda3/envs/DBU/lib/python3.10/site-packages/matplotlib/quiver.py:640: RuntimeWarning: Mean of empty slice.
  amean = a[~self.Umask].mean()
/home/martin/.local/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/martin/anaconda3/envs/DBU/lib/python3.10/site-packages/matplotlib/quiver.py:640: RuntimeWarning: Mean of empty slice.
  amean = a[~self.Umask].mean()
/home/martin/.local/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
